# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [3]:
# Upgrade sklearn to its latest version
!pip install -U scikit-learn

Requirement already up-to-date: scikit-learn in /opt/conda/lib/python3.6/site-packages (0.23.1)


In [4]:
import sklearn
print(sklearn.__version__)

0.23.1


In [5]:
# import libraries

#Basic DS libs
import numpy as np
import pandas as pd

#Helper Libs
import re
from sqlalchemy import create_engine
import joblib

#NLTK
import nltk
from nltk import WordNetLemmatizer
from nltk.corpus import stopwords
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')

#Pipeline
from sklearn.pipeline import Pipeline
from sklearn.pipeline import FeatureUnion
from sklearn.model_selection import train_test_split

#Vectorizers/Transformers
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer, TfidfVectorizer

#Models
from sklearn.multiclass import OneVsRestClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression

#Evaluation Metrics
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import hamming_loss, zero_one_loss, accuracy_score, precision_score, recall_score

#Gensim
from gensim.test.utils import datapath
from gensim import utils
import gensim.models

#Glove
from gensim.test.utils import datapath, get_tmpfile
from gensim.models import KeyedVectors
from gensim.scripts.glove2word2vec import glove2word2vec

#Doc2Vec
from gensim.models.doc2vec import TaggedDocument, Doc2Vec
from sklearn.base import BaseEstimator
import multiprocessing

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [6]:
import sklearn
sklearn.__version__

'0.23.1'

In [7]:
# load data from database
engine = create_engine('sqlite:///DisasterResponse.db')
messages_df = pd.read_sql_table(con=engine, table_name='Message')
categories_df = pd.read_sql_table(con=engine, table_name='CorpusWide')

In [8]:
# create categories_list data with a list of the categories for each message
categories = pd.read_csv('./disaster_categories.csv')
categories['categories_list'] = categories['categories'].apply(lambda x: re.sub(r"[_a-z]+-0[;]?", "", x)) \
                                    .apply(lambda x: re.sub(r"-1", "", x)) \
                                    .apply(lambda x: re.sub(r";$", "", x))

In [9]:
messages_df

,message_id,message,original,genre,num_words
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,13
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,9
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,6
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,13
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,12
5,14,Information about the National Palace-,Informtion au nivaux palais nationl,direct,5
6,15,Storm at sacred heart of jesus,Cyclone Coeur sacr de jesus,direct,6
7,16,"Please, we need tents and water. We are in Sil...",Tanpri nou bezwen tant avek dlo nou zon silo m...,direct,12
8,17,"I would like to receive the messages, thank you",Mwen ta renmen jouin messag yo. Merci,direct,9
9,18,I am in Croix-des-Bouquets. We have health iss...,"Nou kwadebouke, nou gen pwoblem sant m yo nan ...",direct,22


In [10]:
categories_df

,message_id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,14,Information about the National Palace-,Informtion au nivaux palais nationl,direct,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,15,Storm at sacred heart of jesus,Cyclone Coeur sacr de jesus,direct,1,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0
7,16,"Please, we need tents and water. We are in Sil...",Tanpri nou bezwen tant avek dlo nou zon silo m...,direct,1,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,1
8,17,"I would like to receive the messages, thank you",Mwen ta renmen jouin messag yo. Merci,direct,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,18,I am in Croix-des-Bouquets. We have health iss...,"Nou kwadebouke, nou gen pwoblem sant m yo nan ...",direct,1,1,0,1,1,1,...,0,0,0,0,0,0,0,0,0,1


In [11]:
categories

,id,categories,categories_list
0,2,related-1;request-0;offer-0;aid_related-0;medi...,related
1,7,related-1;request-0;offer-0;aid_related-1;medi...,related;aid_related;other_aid;weather_related;...
2,8,related-1;request-0;offer-0;aid_related-0;medi...,related
3,9,related-1;request-1;offer-0;aid_related-1;medi...,related;request;aid_related;medical_products;o...
4,12,related-1;request-0;offer-0;aid_related-0;medi...,related
5,14,related-0;request-0;offer-0;aid_related-0;medi...,
6,15,related-1;request-0;offer-0;aid_related-0;medi...,related;weather_related;storm
7,16,related-1;request-1;offer-0;aid_related-1;medi...,related;request;aid_related;water;shelter;dire...
8,17,related-0;request-0;offer-0;aid_related-0;medi...,
9,18,related-1;request-1;offer-0;aid_related-1;medi...,related;request;aid_related;medical_help;medic...


### 2 - Design and Apply Tokenization Strategies

#### 2.1 - Write down tokenization functions

In [12]:
def tokenize_to_list(text, lemmatizer):
    '''
    INPUT
    text - text string to be tokenized
    lemmatizer - lemmatizer object to be used to process text tokens
    
    OUTPUT
    A list of tokens extracted from the input text
    
    This function receives raw text as input a pre-processes it for NLP analysis, removing punctuation and
    special characters, normalizing case and removing extra spaces, as well as removing stop words and 
    applying lemmatization
    '''
    tokens = nltk.tokenize.word_tokenize(re.sub(r"[^a-zA-Z0-9]", " ", text.lower().strip()))
    clean_tokens = [lemmatizer.lemmatize(tok) for tok in tokens if tok not in stopwords.words("english")]

    return clean_tokens

In [13]:
def tokenize_to_str(text, lemmatizer):
    '''
    INPUT
    text - text string to be tokenized
    lemmatizer - lemmatizer object to be used to process text tokens
    
    OUTPUT
    A string with the tokens extracted from the input text concatenated by spaces
    
    This function receives raw text as input a pre-processes it for NLP analysis, removing punctuation and
    special characters, normalizing case and removing extra spaces, as well as removing stop words and 
    applying lemmatization
    '''
    tokens = nltk.tokenize.word_tokenize(re.sub(r"[^a-zA-Z0-9]", " ", text.lower().strip()))
    clean_tokens = [lemmatizer.lemmatize(tok) for tok in tokens if tok not in stopwords.words("english")]
    #Return tokens list as a string joined by whitespaces
    clean_tokens_str = ' '.join(clean_tokens)

    return clean_tokens_str

In [14]:
lemmatizer = WordNetLemmatizer()
tokenize_to_list(messages_df.message[0],lemmatizer)

['weather', 'update', 'cold', 'front', 'cuba', 'could', 'pas', 'haiti']

In [15]:
tokenize_to_str(messages_df.message[0],lemmatizer)

'weather update cold front cuba could pas haiti'

#### 2.2 - Generate tokenized messages table for further analysis

In [16]:
# %%time
# lemmatizer = WordNetLemmatizer()
# messages_df['tokens_list'] = messages_df.message.apply(lambda x: tokenize_to_list(x, lemmatizer))

In [17]:
# %%time
# messages_df['tokens_str'] = messages_df.message.apply(lambda x: tokenize_to_str(x, lemmatizer))

In [18]:
messages_df

,message_id,message,original,genre,num_words
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,13
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,9
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,6
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,13
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,12
5,14,Information about the National Palace-,Informtion au nivaux palais nationl,direct,5
6,15,Storm at sacred heart of jesus,Cyclone Coeur sacr de jesus,direct,6
7,16,"Please, we need tents and water. We are in Sil...",Tanpri nou bezwen tant avek dlo nou zon silo m...,direct,12
8,17,"I would like to receive the messages, thank you",Mwen ta renmen jouin messag yo. Merci,direct,9
9,18,I am in Croix-des-Bouquets. We have health iss...,"Nou kwadebouke, nou gen pwoblem sant m yo nan ...",direct,22


In [19]:
# messages_tokens = messages_df[['message_id','tokens_str']]

In [20]:
# messages_tokens.to_sql('MessageTokens', engine, index=False, if_exists='replace')

In [21]:
#messages_categories = messages_tokens.merge(categories_df, on='message_id')

In [22]:
#messages_categories.head()

#### 2.4 - Find n-grams and save to DB

In [23]:
# def get_ngrams_freqs(messages_array, n=1):
#     vec = CountVectorizer(ngram_range=(n, n)).fit(messages_array)
#     bag_of_words = vec.transform(messages_array)
#     word_count = bag_of_words.sum(axis=0)
#     words_freq = [(word, n, word_count[0, idx]) for word, idx in vec.vocabulary_.items()]
#     words_freq = sorted(words_freq, key = lambda x: x[2], reverse=True)
#     words_freq_df = pd.DataFrame(data = words_freq, columns = ['ngram','n','count'])
#     return words_freq_df

In [24]:
# unigrams_freqs = get_ngrams_freqs(messages_tokens.tokens_str)
# bigrams_freqs = get_ngrams_freqs(messages_tokens.tokens_str, n=2)
# trigrams_freqs = get_ngrams_freqs(messages_tokens.tokens_str, n=3)
# ngrams_freqs = pd.concat([unigrams_freqs, bigrams_freqs, trigrams_freqs])

In [25]:
# ngrams_freqs

In [26]:
# ngrams_freqs.to_sql('NGramsFreqs', engine, index=False, if_exists='replace')

### 3 - Prepare Data for Train/Test Pipeline
- Extract X and Y from datasets
- Split data into train and test sets
- Train pipeline

In [27]:
messages_tokens = pd.read_sql_table(con=engine, table_name='MessageTokens')

In [28]:
messages_tokens

,message_id,tokens_str
0,2,weather update cold front cuba could pas haiti
1,7,hurricane
2,8,looking someone name
3,9,un report leogane 80 90 destroyed hospital st ...
4,12,say west side haiti rest country today tonight
5,14,information national palace
6,15,storm sacred heart jesus
7,16,please need tent water silo thank
8,17,would like receive message thank
9,18,croix de bouquet health issue worker santo 15 ...


In [29]:
# prepare data for ML Pipeline
X = messages_df.message.values
X_tokenized = messages_tokens.tokens_str.values
Y_df = categories_df.drop(['message_id', 'message', 'original', 'genre'], axis=1)
Y = Y_df.values
category_columns = Y_df.columns

In [30]:
Y_df.sum()

related                   20031
request                    4453
offer                       118
aid_related               10822
medical_help               2078
medical_products           1310
search_and_rescue           723
security                    471
military                    857
water                      1666
food                       2911
shelter                    2303
clothing                    403
money                       602
missing_people              298
refugees                    873
death                      1190
other_aid                  3434
infrastructure_related     1701
transport                  1196
buildings                  1327
electricity                 532
tools                       159
hospitals                   283
shops                       120
aid_centers                 309
other_infrastructure       1147
weather_related            7272
floods                     2142
storm                      2437
fire                        282
earthqua

In [31]:
categories_df

,message_id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,14,Information about the National Palace-,Informtion au nivaux palais nationl,direct,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,15,Storm at sacred heart of jesus,Cyclone Coeur sacr de jesus,direct,1,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0
7,16,"Please, we need tents and water. We are in Sil...",Tanpri nou bezwen tant avek dlo nou zon silo m...,direct,1,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,1
8,17,"I would like to receive the messages, thank you",Mwen ta renmen jouin messag yo. Merci,direct,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,18,I am in Croix-des-Bouquets. We have health iss...,"Nou kwadebouke, nou gen pwoblem sant m yo nan ...",direct,1,1,0,1,1,1,...,0,0,0,0,0,0,0,0,0,1


In [32]:
X

array(['Weather update - a cold front from Cuba that could pass over Haiti',
       'Is the Hurricane over or is it not over',
       'Looking for someone but no name', ...,
       "Proshika, operating in Cox's Bazar municipality and 5 other unions, Ramu and Chokoria, assessment, 5 kg rice, 1,5 kg lentils to 700 families.",
       'Some 2,000 women protesting against the conduct of the elections were teargassed as they tried to converge on the local electoral commission offices in the southern oil city of Port Harcourt.',
       'A radical shift in thinking came about as a result of this meeting, recognizing that HIV/AIDS is at the core of the humanitarian crisis and identifying the crisis itself as a function of the HIV/AIDS pandemic.'],
      dtype=object)

In [33]:
X_tokenized

array(['weather update cold front cuba could pas haiti', 'hurricane',
       'looking someone name', ...,
       'proshika operating cox bazar municipality 5 union ramu chokoria assessment 5 kg rice 1 5 kg lentil 700 family',
       '2 000 woman protesting conduct election teargassed tried converge local electoral commission office southern oil city port harcourt',
       'radical shift thinking came result meeting recognizing hiv aid core humanitarian crisis identifying crisis function hiv aid pandemic'],
      dtype=object)

In [34]:
Y

array([[1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       ...,
       [1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0]])

In [35]:
category_columns

Index(['related', 'request', 'offer', 'aid_related', 'medical_help',
       'medical_products', 'search_and_rescue', 'security', 'military',
       'water', 'food', 'shelter', 'clothing', 'money', 'missing_people',
       'refugees', 'death', 'other_aid', 'infrastructure_related', 'transport',
       'buildings', 'electricity', 'tools', 'hospitals', 'shops',
       'aid_centers', 'other_infrastructure', 'weather_related', 'floods',
       'storm', 'fire', 'earthquake', 'cold', 'other_weather',
       'direct_report'],
      dtype='object')

In [36]:
X_train, X_test, y_train, y_test = train_test_split(X_tokenized, Y, test_size=0.25, random_state=199)

In [37]:
X_train

array(['crude charm charismatic dt member thanks video santiago talk twice',
       'tidal wave triggered earthquake indonesia sunday swept vast swath coastline including india indonesia malaysia maldives somalia sri lanka thailand',
       'earthquake happened 4h34 evening night', ...,
       'earthquake day haiti google possibly leaving china',
       'matter debated previously non aligned movement foreign minister firmly stated right humanitarian intervention',
       'dumbonyc oh rt endtwist coned power station dumbo flooding nopower sandy'],
      dtype=object)

In [38]:
y_train

array([[1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       ...,
       [1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0]])

In [39]:
print(X_train.shape, y_train.shape)

(19608,) (19608, 35)


In [40]:
y_train.sum(axis=0)

array([15027,  3310,    88,  8115,  1570,   982,   538,   354,   669,
        1253,  2161,  1731,   293,   461,   224,   659,   898,  2558,
        1276,   879,  1000,   410,   117,   223,    89,   240,   855,
        5479,  1597,  1818,   214,  1875,   399,  1024,  3791])

___

## Feature Engineering

##### 4 - Design Transformer to Tokenize sentences before passing on to other transformers

In [41]:
class Tokenizer(BaseEstimator):

    def __init__(self, output_str=True):
        self.output_str = output_str
        self.lemmatizer = WordNetLemmatizer()

    def fit(self, X, y):
        return self

    def transform(self, X):
        return np.array([tokenize_to_str(tokens_str, lemmatizer) if self.output_str else 
                         tokenize_to_list(tokens_str, lemmatizer) for tokens_str in X])
            

In [42]:
# %%time
# msgTokenizer = Tokenizer().fit(X, None)
# msgTokenizer.transform(X)

### 4 - Create a Mean/TF-IDF word vector from a locally trained Word2Vec model

#### 4.1 - Train a Word2Vec model using messages text

In [43]:
class Message(object):
    """An interator that yields messages tokens (lists of str)."""
    
    def __init__(self, messages_df, sample_size=-1):
        self.messages_df = messages_df
        self.sample_size = sample_size

    def __iter__(self):
        messages = self.messages_df
        
        #Sample dataset if specified
        if self.sample_size > 0:
            messages = messages.sample(self.sample_size)
        
        #Yeld tokenized message joined by whitespaces
        for token_str in messages.tokens_str:
            yield token_str.split()

In [44]:
messages_tokens.tokens_str

0           weather update cold front cuba could pas haiti
1                                                hurricane
2                                     looking someone name
3        un report leogane 80 90 destroyed hospital st ...
4           say west side haiti rest country today tonight
5                              information national palace
6                                 storm sacred heart jesus
7                        please need tent water silo thank
8                         would like receive message thank
9        croix de bouquet health issue worker santo 15 ...
10                      nothing eat water starving thirsty
11             petionville need information regarding 4636
12       thomassin number 32 area named pyron would lik...
13             let together need food delma 75 didine area
14       information 4636 number order participate see use
15       comitee delmas 19 rue street janvier impasse c...
16       need food water klecin 12 dying hunger impasse.

In [45]:
%%time

sentences = Message(messages_df=messages_tokens)
local_w2v_model_full = gensim.models.Word2Vec(sentences=sentences)
local_w2v_model_full.save("messages-word2vec-full.model")

CPU times: user 7.35 s, sys: 96.3 ms, total: 7.45 s
Wall time: 7.54 s


In [46]:
w2v_model_full = gensim.models.Word2Vec.load("messages-word2vec-full.model")

In [47]:
# look up generated word vocab
for i, word in enumerate(w2v_model_full.wv.vocab):
    if i == 10:
        break
    print(word)

weather
update
cold
front
cuba
could
pas
haiti
hurricane
looking


In [48]:
# look up generated word vectors
w2v_model_full.wv['earthquake']

array([ 0.19078784, -0.18491617,  0.03814096,  0.3418014 , -0.9759837 ,
       -1.4897901 , -0.43548572, -0.20545296,  1.2964498 ,  0.52204883,
        0.06461112,  0.13435331, -0.4328741 , -0.9239483 , -0.22408496,
       -0.5571155 ,  0.43278468,  0.55698615, -0.1026466 , -0.49350274,
       -0.24672078,  0.888357  ,  0.73535264, -0.44993743,  1.2125608 ,
       -0.8714119 ,  0.3708165 ,  0.04596617, -0.19340983,  0.09381265,
        0.07652403,  0.19241536,  0.47503707,  0.0617723 , -0.6348238 ,
       -0.02342288,  0.6671087 ,  0.07306863,  0.9675678 ,  0.8765735 ,
       -1.0461487 ,  0.47872558, -0.3998758 , -1.3513135 ,  0.24322149,
       -0.13318177, -0.12466659,  0.15902896, -0.9197659 ,  0.4422364 ,
       -0.74258924,  1.3287657 , -0.58001095,  0.53123295, -0.72101235,
       -0.5474531 , -0.5145969 ,  0.56529915, -0.33326197, -0.00676469,
        0.54244834, -0.5966822 , -0.98309493,  0.21653189,  0.8753762 ,
        0.0968083 , -1.0476868 ,  0.5039579 ,  0.47279334,  0.37

#### 4.2 - Implement Vector Aggregator

Inspecting variables

In [49]:
# len(X)

In [50]:
# X[0]

In [51]:
# X.shape[0]

Initial implementation of aggregators

In [52]:
from collections import defaultdict

class MeanEmbeddingVectorizer(BaseEstimator):
    def __init__(self, word2vec_model):
        self.word2vec_model = word2vec_model
        self.num_dims = word2vec_model.vector_size
            
    def fit(self, X, y):
        return self 

    def transform(self, X):
        mean_embeddings = np.empty([X.shape[0],self.num_dims])
        
        for i in range(X.shape[0]):
            doc_tokens = X[i]
            
            words_vectors_concat = [self.word2vec_model[w] for w in doc_tokens if w in self.word2vec_model]

            if (len(words_vectors_concat) == 0):
                words_vectors_concat = [np.zeros(self.num_dims)]
                
            #print(np.mean(words_vectors_concat, axis=0))
                
            mean_embeddings[i] = np.mean(words_vectors_concat, axis=0)
            
        return mean_embeddings

In [53]:
# #Testing defaultdict functionality
# test_dict = defaultdict(lambda: 1, [("Hello" , 7), ("hi" , 10), ("there" , 45),("at" , 23),("this" , 77)])
# test_dict['Hi']

In [54]:
class TfidfEmbeddingVectorizer(BaseEstimator):
    def __init__(self, word2vec_model):
        self.word2vec_model = word2vec_model
        self.num_dims = word2vec_model.vector_size
        self.word_weights = None
        
    def fit(self, X, y):
        tfidf = TfidfVectorizer(analyzer=lambda x: x)
        tfidf.fit(X)
        max_idf = max(tfidf.idf_)
        
        tfidf_weights = [(w, tfidf.idf_[i]) for w, i in tfidf.vocabulary_.items()]
        self.word_weights = defaultdict(lambda: max_idf, tfidf_weights)
    
        return self
    
    def transform(self, X):
        mean_embeddings = np.empty([X.shape[0],self.num_dims])
        
        for i in range(X.shape[0]):
            doc_tokens = X[i]
            
            words_vectors_concat = [self.word2vec_model[w]*self.word_weights[w] for w in doc_tokens if w in self.word2vec_model]

            if (len(words_vectors_concat) == 0):
                words_vectors_concat = [np.zeros(self.num_dims)]
                
            #print(np.mean(words_vectors_concat, axis=0))
                
            mean_embeddings[i] = np.mean(words_vectors_concat, axis=0)
            
        return mean_embeddings

Test aggregators using a Pipeline

In [55]:
# %%time

# w2v_mean_pip = Pipeline([
#     ('features',FeatureUnion([
#         ('w2v_mean', MeanEmbeddingVectorizer(w2v_model_full)),
#         ('w2v_tfidf', TfidfEmbeddingVectorizer(w2v_model_full))
#     ])),    
#     ('clf', MultiOutputClassifier(GaussianNB()))
# ])

# w2v_mean_pip.fit(X_train, y_train)

# y_pred = w2v_mean_pip.predict(X_test)

# for category_idx in range(y_pred.shape[1]):
#     print(classification_report(y_pred=y_pred[:,category_idx],y_true=y_test[:,category_idx], labels=[0,1], target_names=[category_columns[category_idx] + '-0',category_columns[category_idx] + '-1']))

In [56]:
# category_idx=1
# c_report = classification_report(y_pred=y_pred[:,category_idx],y_true=y_test[:,category_idx], 
#                       labels=[0,1], 
#                       target_names=[category_columns[category_idx] + '-0',category_columns[category_idx] + '-1'],
#                       output_dict=True)
# c_report

In [57]:
# print(c_report.keys())

In [58]:
# w2v_mean_pip.score(X_test, y_test)

In [59]:
# y_pred[0]

y_pred.shape

y_test[0]

y_test.shape

sum(y_pred[0] == y_test[0])/y_pred[0].shape[0]

sample_test = y_test[0]
sample_pred = y_pred[0]

sample_test == 1

sample_pred == 1

(sample_test == 1) == (sample_pred == 1)

#Computing TPR manually
tp_array = (sample_test == 1) & (sample_pred == 1)
tp = tp_array.sum()
tpr = tp/sample_test.shape[0]
print(tp)
print(tpr)
tp_array

3/36

#Computing FPR manually
fp_array = (sample_test == 0) & (sample_pred == 1)
fp = fp_array.sum()
fpr = fp/sample_test.shape[0]
print(fp)
print(fpr)
fp_array

#Computing FNR manually
fn_array = (sample_test == 1) & (sample_pred == 0)
fn = fn_array.sum()
fnr = fn/sample_test.shape[0]
print(fn)
print(fnr)
fn_array

#Computing Precision manually
print(tp/(tp+fp))

#Computing Recall manually
print(tp/(tp+fn))

example_test = np.append(np.repeat(1, 4),np.repeat(0,32))
example_test

example_pred = np.repeat(0, 36)
example_pred

from sklearn.metrics import f1_score

print("Hamming Loss = ", hamming_loss(example_test,example_pred))
print("Zero-One Loss = ", zero_one_loss(example_test,example_pred))
print("Jaccard Score = ", jaccard_score(example_test,example_pred))
print("Accuracy = ", accuracy_score(example_test,example_pred))
print("Recall = ", recall_score(example_test,example_pred))
print("Precision = ", precision_score(example_test,example_pred))
print("F1-Score = ", f1_score(example_test,example_pred))
print("Jaccard Score Micro = ", jaccard_score(example_test,example_pred, average='micro'))
print("Recall Micro = ", recall_score(example_test,example_pred, average='micro'))
print("Precision Micro = ", precision_score(example_test,example_pred, average='micro'))
print("F1-Score Micro = ", f1_score(example_test,example_pred, average='micro'))

print("Hamming Loss = ", hamming_loss(np.argmax(y_test, axis=1),np.argmax(y_pred, axis=1)))
print("Zero-One Loss = ", zero_one_loss(np.argmax(y_test, axis=1),np.argmax(y_pred, axis=1)))
print("Jaccard Score = ", jaccard_score(np.argmax(y_test, axis=1),np.argmax(y_pred, axis=1), average='micro'))
print("Accuracy = ", accuracy_score(np.argmax(y_test, axis=1),np.argmax(y_pred, axis=1)))
print("Recall = ", recall_score(np.argmax(y_test, axis=1),np.argmax(y_pred, axis=1), average='micro'))
print("Precision = ", precision_score(np.argmax(y_test, axis=1),np.argmax(y_pred, axis=1), average='micro'))

from sklearn.metrics import mean_absolute_error, r2_score, explained_variance_score
print(mean_absolute_error(y_test, y_pred, multioutput='raw_values'))
print(mean_absolute_error(y_test, y_pred))
(y_test == y_pred).mean()
print(r2_score(y_test, y_pred))
print(r2_score(y_test, y_pred, multioutput='variance_weighted'))
print(explained_variance_score(y_test, y_pred))
print(explained_variance_score(y_test, y_pred, multioutput='variance_weighted'))

#### 4.3 - Create Estimator to both train and average w2v on corpus data

Trying W2VTransformer - Gensim's sklearn Transformer

from gensim.sklearn_api import W2VTransformer
from gensim.test.utils import common_texts

model = W2VTransformer(size=10, min_count=1, seed=1)

model.fit(common_texts)
          
model.transform(['graph', 'system'])

common_texts

messages_df.head(5).tokens_str.str.split().values

messages_df.head(5).tokens_str

w2v_model = W2VTransformer()
w2v_model.fit(messages_df.tokens_str.str.split().values)

word = messages_df.tokens_str[0].split()[1]
print(word)
w2v_model.transform(word)

test_model = gensim.models.Word2Vec(messages_df.tokens_str.str.split().values)

for i, word in enumerate(test_model.wv.vocab):
    if i == 10:
        break
    print(word)

test_model.wv['earthquake']

Re-implement aggregators now with training capability

Test aggregators using a Pipeline

In [60]:
# %%time

# from sklearn.naive_bayes import GaussianNB
# from sklearn.metrics import classification_report

# w2v_mean_pip = Pipeline([
#     ('features',FeatureUnion([
#         ('w2v_mean', MeanEmbeddingTrainVectorizer(word2vec_model=w2v_model_full)),
#         ('w2v_mean_train', MeanEmbeddingTrainVectorizer(num_dims=50)),
#         ('w2v_tfidf', TfidfEmbeddingTrainVectorizer(word2vec_model=w2v_model_full)),
#         ('w2v_tfidf_train', TfidfEmbeddingTrainVectorizer(num_dims=100)),
#     ])),
#     ('clf', MultiOutputClassifier(GaussianNB()))
# ])

# w2v_mean_pip.fit(X_train, y_train)

# y_pred = w2v_mean_pip.predict(X_test)

# for category_idx in range(y_pred.shape[1]):
#     print(classification_report(y_pred=y_pred[:,category_idx],y_true=y_test[:,category_idx], labels=[0,1], target_names=[category_columns[category_idx] + '-0',category_columns[category_idx] + '-1']))

### 5. Use Glove pre-trained model to generate feature vectors

Glove pre-trained vectors can be downloaded here: https://nlp.stanford.edu/projects/glove/

%%time

# Code which can be used to convert from Glove default format to Gensim W2V format

glove_file = 'glove-pretrained/glove.6B.50d.txt'
tmp_file = 'glove-pretrained/glove.6B.50d_word2vec.txt'

#_ = glove2word2vec(glove_file, tmp_file)

glove_model = KeyedVectors.load_word2vec_format(tmp_file)

In [61]:
%%time

from gensim.models import KeyedVectors

glove_50d_w2v = KeyedVectors.load_word2vec_format('glove-pretrained/glove.6B.50d_word2vec.txt', binary=False)

CPU times: user 28.1 s, sys: 450 ms, total: 28.5 s
Wall time: 29.4 s


In [62]:
%%time

glove_100d_w2v = KeyedVectors.load_word2vec_format('glove-pretrained/glove.6B.100d_word2vec.txt', binary=False)

CPU times: user 52.3 s, sys: 675 ms, total: 52.9 s
Wall time: 53.4 s


In [63]:
%%time

glove_300d_w2v = KeyedVectors.load_word2vec_format('glove-pretrained/glove.6B.300d_word2vec.txt', binary=False)

CPU times: user 2min 28s, sys: 1.88 s, total: 2min 30s
Wall time: 2min 32s


##### Use a Pipeline to train the Word2Vec estimator

Test Aggregators with Glove using a Pipeline

%%time

from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import classification_report

w2v_mean_pip = Pipeline([
    ('features',FeatureUnion([
        ('glove_mean', MeanEmbeddingTrainVectorizer(glove_300d_w2v)),
        ('glove_tfidf', TfidfEmbeddingTrainVectorizer(glove_300d_w2v)),
    ])),    
    ('clf', MultiOutputClassifier(GaussianNB()))
])

w2v_mean_pip.fit(X_train, y_train)

y_pred = w2v_mean_pip.predict(X_test)

for category_idx in range(y_pred.shape[1]):
    print(classification_report(y_pred=y_pred[:,category_idx],y_true=y_test[:,category_idx], labels=[0,1], target_names=[category_columns[category_idx] + '-0',category_columns[category_idx] + '-1']))
    
print("y_pred sum = ",y_pred.sum())
print("Hamming Loss = ", hamming_loss(np.argmax(y_test, axis=1),np.argmax(y_pred, axis=1)))
print("Zero-One Loss = ", zero_one_loss(np.argmax(y_test, axis=1),np.argmax(y_pred, axis=1)))
print("Jaccard Score = ", jaccard_score(np.argmax(y_test, axis=1),np.argmax(y_pred, axis=1), average='micro'))
print("Accuracy = ", accuracy_score(np.argmax(y_test, axis=1),np.argmax(y_pred, axis=1)))

In [64]:
#from sklearn.metrics import precision_recall_curve
#from sklearn.metrics import average_precision_score

# For each class
# precision = dict()
# recall = dict()
# average_precision = dict()
# for i in range(len(category_columns)):
#     precision[i], recall[i], _ = precision_recall_curve(y_test[:, i], y_pred[:, i])
    #average_precision[i] = average_precision_score(y_test[:, i], y_pred[:, i])

# A "micro-average": quantifying score on all classes jointly
# precision["micro"], recall["micro"], _ = precision_recall_curve(Y_test.ravel(),
#     y_score.ravel())
# average_precision["micro"] = average_precision_score(Y_test, y_score,
#                                                      average="micro")
# print('Average precision score, micro-averaged over all classes: {0:0.2f}'
#       .format(average_precision["micro"]))

from sklearn.metrics import hamming_loss, zero_one_loss, jaccard_score, accuracy_score, auc, recall_score, precision_score

print("Hamming Loss = ", hamming_loss(np.argmax(y_test, axis=1),np.argmax(y_pred, axis=1)))
print("Zero-One Loss = ", zero_one_loss(np.argmax(y_test, axis=1),np.argmax(y_pred, axis=1)))
print("Jaccard Score = ", jaccard_score(np.argmax(y_test, axis=1),np.argmax(y_pred, axis=1), average='micro'))
print("Accuracy = ", accuracy_score(np.argmax(y_test, axis=1),np.argmax(y_pred, axis=1)))
print("Recall = ", recall_score(np.argmax(y_test, axis=1),np.argmax(y_pred, axis=1), average='micro'))
print("Precision = ", precision_score(np.argmax(y_test, axis=1),np.argmax(y_pred, axis=1), average='micro'))

y_test[0] == y_pred[0]

y_test[0]

y_pred[0]

6536*36

print(np.argmax(y_pred, axis=1).shape)
print(y_pred.shape)

from sklearn.preprocessing import MultiLabelBinarizer
train_foo = [['sci-fi', 'thriller'],['comedy'],['sci-fi', 'thriller'],['comedy']]
mlb = MultiLabelBinarizer()
mlb_label_train = mlb.fit_transform(train_foo)
mlb_label_train

y_test

y_pred

Y_df[Y_df['related'] == 2]

### 6. Use Doc2Vec to generate feature vectors

# Checking the number of words in the corpus
messages_df['message'].apply(lambda x: len(x.split(' '))).sum()

Try Doc2Vec step by step

def read_messages(messages, tokens_only=False):
    for index, tokens_str in np.ndenumerate(messages):
        tokens = tokens_str.split()
        if tokens_only:
            yield tokens
        else:
            # For training data, add tags
            tags = index
            yield gensim.models.doc2vec.TaggedDocument(tokens, tags)

train_corpus = list(read_messages(X_train))
test_corpus = list(read_messages(X_test, tokens_only=True))

train_corpus[:10]

test_corpus[:10]

d2v_model = gensim.models.doc2vec.Doc2Vec(vector_size=50, min_count=5, epochs=10)

d2v_model.build_vocab(train_corpus)

%%time 

d2v_model.train(train_corpus, total_examples=d2v_model.corpus_count, epochs=d2v_model.epochs)

#### Create a Doc2Vec Estimator/Transformer 

In [65]:
class Doc2VecTransformer(BaseEstimator):

    def __init__(self, vector_size=100, epochs=20):
        self.epochs = epochs
        self.vector_size = vector_size
        self.workers = multiprocessing.cpu_count() - 1
        self.d2v_model = None

    def fit(self, X, y):
        tagged_x = [TaggedDocument(tokens_str.split(), [index]) for index, tokens_str in np.ndenumerate(X)]
        self.d2v_model = Doc2Vec(vector_size=self.vector_size, workers=self.workers, epochs=self.epochs)
        self.d2v_model.build_vocab(tagged_x)
        self.d2v_model.train(tagged_x, total_examples=self.d2v_model.corpus_count, 
                             epochs=self.epochs)

        return self

    def transform(self, X):
        return np.asmatrix(np.array([self.d2v_model.infer_vector(tokens_str.split())
                                     for tokens_str in X]))

%%time

doc2vec_trf = Doc2VecTransformer(epochs=1)
doc2vec_trf.fit(X_train, y_train)

X_test[0].split()

inferred_vector = doc2vec_trf.d2v_model.infer_vector(X_test[0].split())
print(inferred_vector.shape)
print(type(inferred_vector))
print(inferred_vector)

inferred_vector2 = np.asmatrix(np.array([doc2vec_trf.d2v_model.infer_vector(X_test[0].split())]))
print(inferred_vector2.shape)
print(type(inferred_vector2))
print(inferred_vector2)

X_train

X_test

doc2vec_features = doc2vec_trf.transform(X_test)
print(doc2vec_features.shape)
doc2vec_features

Test Doc2Vec using a Pipeline

%%time

from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import classification_report

w2v_mean_pip = Pipeline([
    ('doc2vec', Doc2VecTransformer()),
    ('clf', MultiOutputClassifier(GaussianNB()))
])

w2v_mean_pip.fit(X_train, y_train)

y_pred = w2v_mean_pip.predict(X_test)

for category_idx in range(y_pred.shape[1]):
    print(classification_report(y_pred=y_pred[:,category_idx],y_true=y_test[:,category_idx], labels=[0,1], target_names=[category_columns[category_idx] + '-0',category_columns[category_idx] + '-1']))
    
print("y_pred sum = ",y_pred.sum())

### 7. Extra Feature - Distance Between Message Word Vector to Categories Word Vectors 

We will use Glove as our space vectorizer as it is trained on a broad corpus and thus able to better describe texts / words

First, let's build the embeddings for the categories

In [66]:
category_columns

Index(['related', 'request', 'offer', 'aid_related', 'medical_help',
       'medical_products', 'search_and_rescue', 'security', 'military',
       'water', 'food', 'shelter', 'clothing', 'money', 'missing_people',
       'refugees', 'death', 'other_aid', 'infrastructure_related', 'transport',
       'buildings', 'electricity', 'tools', 'hospitals', 'shops',
       'aid_centers', 'other_infrastructure', 'weather_related', 'floods',
       'storm', 'fire', 'earthquake', 'cold', 'other_weather',
       'direct_report'],
      dtype='object')

In [67]:
categories_tokens = np.array([np.array(cat.split('_')) for cat in category_columns])

In [68]:
categories_tokens

array([array(['related'], dtype='<U7'), array(['request'], dtype='<U7'),
       array(['offer'], dtype='<U5'),
       array(['aid', 'related'], dtype='<U7'),
       array(['medical', 'help'], dtype='<U7'),
       array(['medical', 'products'], dtype='<U8'),
       array(['search', 'and', 'rescue'], dtype='<U6'),
       array(['security'], dtype='<U8'), array(['military'], dtype='<U8'),
       array(['water'], dtype='<U5'), array(['food'], dtype='<U4'),
       array(['shelter'], dtype='<U7'), array(['clothing'], dtype='<U8'),
       array(['money'], dtype='<U5'),
       array(['missing', 'people'], dtype='<U7'),
       array(['refugees'], dtype='<U8'), array(['death'], dtype='<U5'),
       array(['other', 'aid'], dtype='<U5'),
       array(['infrastructure', 'related'], dtype='<U14'),
       array(['transport'], dtype='<U9'),
       array(['buildings'], dtype='<U9'),
       array(['electricity'], dtype='<U11'),
       array(['tools'], dtype='<U5'), array(['hospitals'], dtype='<U9'),
   

glove_300d_w2v_mean = MeanEmbeddingTrainVectorizer(glove_300d_w2v)

glove_300d_w2v_mean.fit(categories_tokens, None)

cat_tokens_embeddings = glove_300d_w2v_mean.transform(categories_tokens)

print(cat_tokens_embeddings.shape)
print(cat_tokens_embeddings)

cat_tokens_embeddings[0]

test_msgs_embeddings = glove_300d_w2v_mean.transform(X_test)

print(test_msgs_embeddings.shape)
print(test_msgs_embeddings)

test_msgs_embeddings[0]

from sklearn.metrics.pairwise import cosine_similarity

print(test_msgs_embeddings[:10].shape)
print(cat_tokens_embeddings.shape)
print(cosine_similarity(test_msgs_embeddings[:10],cat_tokens_embeddings).shape)

#### Create a CategoriesSimilarity Transformer 

Test the transformer manually using test data

In [69]:
# cats_sim_transf = CategoriesSimilarity(categories_tokens, glove_50d_w2v)

In [70]:
# cats_sim_transf.fit(X_test, None)

In [71]:
# cats_sim_transf.transform(X_test)

Test CategoriesSimilarity feature using a Pipeline

%%time

from sklearn.pipeline import FeatureUnion

cats_sim_pip = Pipeline([
    ('cats_sim', CategoriesSimilarity(glove_300d_w2v, cat_tokens_embeddings)),
    ('clf', MultiOutputClassifier(GaussianNB()))
])

cats_sim_pip.fit(X_train, y_train)

y_pred = cats_sim_pip.predict(X_test)

for category_idx in range(y_pred.shape[1]):
    print(classification_report(y_pred=y_pred[:,category_idx],y_true=y_test[:,category_idx], labels=[0,1], target_names=[category_columns[category_idx] + '-0',category_columns[category_idx] + '-1']))
    
print("Score = ", cats_sim_pip.score(X_test, y_test))

# accuracy
print("Accuracy = ",accuracy_score(y_test,y_pred))
print("\n")

### 8. NLP Feature Selection using Sklearn Pipelines

#### 8.1 - All features together

%%time

from sklearn.pipeline import FeatureUnion

all_feats = Pipeline([
    ('features', FeatureUnion([
        ('local_w2v_tfidf', TfidfEmbeddingTrainVectorizer()),
        ('glove_tfidf', TfidfEmbeddingTrainVectorizer(glove_300d_w2v)),
        ('doc2vec', Doc2VecTransformer()),
        ('cats_sim', CategoriesSimilarity(glove_300d_w2v, categories_tokens))
    ])),
    ('clf', MultiOutputClassifier(GaussianNB()))
])

all_feats.fit(X_train, y_train)

y_pred = all_feats.predict(X_test)

for category_idx in range(y_pred.shape[1]):
    print(classification_report(y_pred=y_pred[:,category_idx],y_true=y_test[:,category_idx], labels=[0,1], target_names=[category_columns[category_idx] + '-0',category_columns[category_idx] + '-1']))

score = all_feats.score(X_test, y_test)

score

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

pipeline = Pipeline([
    ('vect',CountVectorizer(tokenizer=tokenize_to_list(lemmatizer=lemmatizer), ngram_range=(1,3))),
    ('tfidf', TfidfTransformer()),
    ('multi_clf', MultiOutputClassifier(RandomForestClassifier(random_state=199), n_jobs=-1))
])

In [72]:
# pipeline.fit(X_train, y_train)

#### Build pipelines for different combinations of features and classifiers

from sklearn import svm

local_w2v_tfidf_svc = Pipeline([
    ('local_w2v_tfidf', TfidfEmbeddingTrainVectorizer()),
    ('clf', MultiOutputClassifier(svm.LinearSVC(random_state=199)))
])

local_w2v_tfidf_rf = Pipeline([
    ('local_w2v_tfidf', TfidfEmbeddingTrainVectorizer()),
    ('clf', MultiOutputClassifier(RandomForestClassifier(random_state=199)))
])

glove_tfidf_svc = Pipeline([
    ('glove_tfidf', TfidfEmbeddingTrainVectorizer(glove_300d_w2v)),
    ('clf', MultiOutputClassifier(svm.LinearSVC(random_state=199)))
])

glove_tfidf_rf = Pipeline([
    ('glove_tfidf', TfidfEmbeddingTrainVectorizer(glove_300d_w2v)),
    ('clf', MultiOutputClassifier(RandomForestClassifier(random_state=199)))
])

doc2vec_svc = Pipeline([
    ('doc2vec', Doc2VecTransformer()),
    ('clf', MultiOutputClassifier(svm.LinearSVC(random_state=199)))
])

doc2vec_rf = Pipeline([
    ('doc2vec', Doc2VecTransformer()),
    ('clf', MultiOutputClassifier(RandomForestClassifier(random_state=199)))
])

all_feats_svc = Pipeline([
    ('features', FeatureUnion([
        ('local_w2v_tfidf', TfidfEmbeddingTrainVectorizer()),
        ('glove_tfidf', TfidfEmbeddingTrainVectorizer(glove_300d_w2v)),
        ('doc2vec', Doc2VecTransformer()),
        ('cats_sim', CategoriesSimilarity(glove_300d_w2v, categories_tokens))
    ])),
    ('clf', MultiOutputClassifier(svm.LinearSVC(random_state=199)))
])

all_feats_rf = Pipeline([
    ('features', FeatureUnion([
        ('local_w2v_tfidf', TfidfEmbeddingTrainVectorizer()),
        ('glove_tfidf', TfidfEmbeddingTrainVectorizer(glove_300d_w2v)),
        ('doc2vec', Doc2VecTransformer()),
        ('cats_sim', CategoriesSimilarity(glove_300d_w2v, categories_tokens))
    ])),
    ('clf', MultiOutputClassifier(RandomForestClassifier(random_state=199)))
])

pipelines = [local_w2v_tfidf_svc, local_w2v_tfidf_rf, glove_tfidf_svc, glove_tfidf_rf, 
             doc2vec_svc, doc2vec_rf, all_feats_svc, all_feats_rf]

grid_dict = {0 : 'Local Word2Vec - TF-IDF - Linear SVC',
             1 : 'Local Word2Vec - TF-IDF - Random Forest',
             2 : 'Glove 300d - TF-IDF - Linear SVC',
             3 : 'Glove 300d - TF-IDF - Random Forest',
             4 : 'Doc2Vec - Linear SVC',
             5 : 'Doc2Vec - Random Forest',
             6 : 'All Features - Linear SVC',
             7 : 'All Features - Random Forest',}

In [73]:
# param_grid = [
#     {'features' : [MeanEmbeddingTrainVectorizer(),TfidfEmbeddingTrainVectorizer()],
#      'features__word2vec_model' : [None,glove_50d_w2v,glove_100d_w2v,glove_300d_w2v],
#      'features__num_dims' : [50,100,300]},
#     {'features' : [Doc2VecTransformer()],
#      'features__vector_size' : [50,100,300]},
#     {'features' : [CategoriesSimilarity(glove_50d_w2v,categories_tokens)],
#      'features__word2vec_model' : [glove_50d_w2v,glove_100d_w2v,glove_300d_w2v]},
#     {'features' : [MeanEmbeddingTrainVectorizer(),TfidfEmbeddingTrainVectorizer()],
#      'features__word2vec_model' : [None,glove_50d_w2v,glove_100d_w2v,glove_300d_w2v],
#      'features__num_dims' : [50,100,300]},
#     {'classifier' : [MultiOutputClassifier(LogisticRegression(random_state=199))],
#      'classifier__estimator__penalty' : ['l1', 'l2'],
#      'classifier__estimator__C' : [0.1, 1, 3],
#      'classifier__estimator__solver' : ['liblinear']},
#     {'classifier' : [MultiOutputClassifier(RandomForestClassifier(random_state=199))],
#     'classifier__estimator__n_estimators' : [10,50,250],
#     'classifier__estimator__max_depth' : [10, 50, 100],
#     'classifier__estimator__min_samples_split' : [1, 5, 10]}
# ]

In [74]:
# generic_pip = Pipeline([
#     ('features',TfidfEmbeddingTrainVectorizer()),
#     ('classifier',MultiOutputClassifier(RandomForestClassifier(random_state=199)))
# ])

In [75]:
# MeanEmbeddingTrainVectorizer().get_params().keys()

In [76]:
# param_grid = [
#     {'features' : [TfidfEmbeddingTrainVectorizer()],
#      'features__word2vec_model' : [None,glove_50d_w2v],
#      'features__num_dims' : [50]}]

In [77]:
# from sklearn.model_selection import RandomizedSearchCV

# #score = make_scorer(mean_absolute_error, greater_is_better=False)
# jobs = -1
# score = 'f1_micro'
# def_cv = 3

# generic_gs = GridSearchCV(estimator=generic_pip,
#             param_grid=param_grid,
#             scoring=score,
#             cv=def_cv,
#             n_jobs=jobs,
#             verbose=True)  

# # randomized_gs = RandomizedSearchCV(estimator=generic_pip,
# #             param_distributions=param_grid,
# #             scoring=score,
# #             cv=def_cv,
# #             n_jobs=jobs,
# #             verbose=True,
# #             random_state=199,
# #             n_iter=2)  

In [78]:
# %%time

# grids = [generic_gs]

# grid_dict = {
#     0 : 'Generic Grid',
# }

# print('Performing Grid Search...')
# # Fit grid search
# generic_gs.fit(X_train, y_train)
# # Best params
# print('Best params: %s' % generic_gs.best_params_)
# # Best training data accuracy
# print('Best training score: %.3f' % generic_gs.best_score_)
# # Predict on test data with best params
# test_score = generic_gs.score(X_test, y_test)
# # Test data accuracy of model with best params
# print('Test set score for best params: %.3f ' % test_score)

# # Track Grid Search results
# gs_results_df = pd.DataFrame(generic_gs.cv_results_)
# gs_results_df['best_model_test_score'] = test_score

#### Run a RandomizedSearchCV for each featureset

In [79]:
class MeanEmbeddingTrainVectorizer(BaseEstimator):

    def __init__(self, word2vec_model=None, num_dims=100):
        if word2vec_model is None:
            self.word2vec_model = None
            self.num_dims = num_dims
            self.workers = multiprocessing.cpu_count() - 1
            
        else:
            self.word2vec_model = word2vec_model
            self.num_dims = word2vec_model.vector_size
            
        print(self.num_dims)
            
    def fit(self, X, y):
        if self.word2vec_model is None:
            self.word2vec_model = gensim.models.Word2Vec(X, size=self.num_dims, 
                                                         workers=self.workers)
        
        return self 

    def transform(self, X):
        mean_embeddings = np.empty([X.shape[0],self.num_dims])
        
        for i in range(X.shape[0]):
            doc_tokens = X[i]
            
            words_vectors_concat = [self.word2vec_model[w] for w in doc_tokens if w in self.word2vec_model]

            if (len(words_vectors_concat) == 0):
                words_vectors_concat = [np.zeros(self.num_dims)]
                
            #print(np.mean(words_vectors_concat, axis=0))
                
            mean_embeddings[i] = np.mean(words_vectors_concat, axis=0)
            
        return mean_embeddings

In [80]:
class TfidfEmbeddingTrainVectorizer(BaseEstimator):
    
    def __init__(self, word2vec_model=None, num_dims=100):
        self.word2vec_model = word2vec_model
        self.num_dims = num_dims
        
    def fit(self, X, y):
        if self.word2vec_model is None:
            self.workers_ = multiprocessing.cpu_count() - 1
            self.word2vec_model = gensim.models.Word2Vec(X, size=self.num_dims, 
                                                         workers=self.workers_)
        self.num_dims = self.word2vec_model.vector_size
            
        tfidf = TfidfVectorizer(analyzer=lambda x: x)
        tfidf.fit(X)
        max_idf = max(tfidf.idf_)
        
        tfidf_weights = [(w, tfidf.idf_[i]) for w, i in tfidf.vocabulary_.items()]
        self.word_weights_ = defaultdict(lambda: max_idf, tfidf_weights)
    
        return self
    
    def transform(self, X):
        mean_embeddings = np.empty([X.shape[0],self.num_dims])
        
        for i in range(X.shape[0]):
            doc_tokens = X[i]
            
            words_vectors_concat = [self.word2vec_model[w]*self.word_weights_[w] for w in doc_tokens if w in self.word2vec_model]

            if (len(words_vectors_concat) == 0):
                words_vectors_concat = [np.zeros(self.num_dims)]
                
            #print(np.mean(words_vectors_concat, axis=0))
                
            mean_embeddings[i] = np.mean(words_vectors_concat, axis=0)
            
        return mean_embeddings

In [81]:
from sklearn.metrics.pairwise import cosine_similarity

class CategoriesSimilarity(BaseEstimator):
    
    def __init__(self, categories_tokens, word2vec_model=None, num_dims=100):
        self.categories_tokens = categories_tokens
        self.word2vec_model = word2vec_model    
        self.num_dims = num_dims
        
    def compute_mean_embeddings(self, tokens_array):    
        mean_embeddings = np.empty([tokens_array.shape[0],self.num_dims])
        
        for i in range(tokens_array.shape[0]):
            doc_tokens = tokens_array[i]
            
            words_vectors_concat = [self.word2vec_model[w] for w in doc_tokens if w in self.word2vec_model]

            if (len(words_vectors_concat) == 0):
                words_vectors_concat = [np.zeros(self.num_dims)]
                
            #print(np.mean(words_vectors_concat, axis=0))
                
            mean_embeddings[i] = np.mean(words_vectors_concat, axis=0)
            
        return mean_embeddings
                    
    def fit(self, X, y):
        if self.word2vec_model is None:
            self.workers_ = multiprocessing.cpu_count() - 1
            self.word2vec_model = gensim.models.Word2Vec(X, size=self.num_dims, 
                                                         workers=self.workers_)
        self.num_dims = self.word2vec_model.vector_size        
        self.categories_vectors_ = self.compute_mean_embeddings(self.categories_tokens)
        return self 

    def transform(self, X):
        mean_embeddings = self.compute_mean_embeddings(X)
        cats_similarities = cosine_similarity(mean_embeddings, self.categories_vectors_)
            
        return cats_similarities

##### Define Pipelines for each featureset

In [89]:
pip_local_w2v = Pipeline([
    ('local_w2v', TfidfEmbeddingTrainVectorizer()),
    ('clf', MultiOutputClassifier(RandomForestClassifier()))
])

pip_glove = Pipeline([
    ('glove', TfidfEmbeddingTrainVectorizer(glove_50d_w2v)),
    ('clf', MultiOutputClassifier(RandomForestClassifier()))
])

pip_doc2vec = Pipeline([
    ('doc2vec', Doc2VecTransformer()),
    ('clf', MultiOutputClassifier(RandomForestClassifier()))
])

pip_cats_sim = Pipeline([
    ('cats_sim', CategoriesSimilarity(categories_tokens=categories_tokens)),
    ('clf', MultiOutputClassifier(RandomForestClassifier()))
])

##### Define Grid Params for each featureset / classifier

In [111]:
params_options_models_simple = [MultiOutputClassifier(RandomForestClassifier(random_state=199,
                                                        n_estimators=50,
                                                        max_depth=100,
                                                        min_samples_split=5)),
                                            MultiOutputClassifier(LogisticRegression(random_state=199,
                                                                                    solver='liblinear',
                                                                                    C=1,
                                                                                    penalty='l2'))]

params_options_local_w2v_simple = {'local_w2v__num_dims' : [50,100],
                                   'clf' : params_options_models_simple}

params_options_local_w2v = {'local_w2v__num_dims' : [50,100,300],
                            'clf' : params_options_models_simple}

params_options_glove = {'glove__word2vec_model' : [glove_50d_w2v,glove_100d_w2v,glove_300d_w2v],
                            'clf' : params_options_models_simple}

params_options_doc2vec = {'doc2vec__vector_size' : [50,100,300],
                            'clf' : params_options_models_simple}

params_options_cat_sim = {'cats_sim__word2vec_model' : [glove_50d_w2v,glove_100d_w2v,glove_300d_w2v],
                            'clf' : params_options_models_simple}

params_options_rf = {'clf' : [MultiOutputClassifier(RandomForestClassifier(random_state=199))],
                     'clf__estimator__n_estimators' : [50,250],
                     'clf__estimator__max_depth' : [50,100],
                     'clf__estimator__min_samples_split' : [2, 5]}

params_options_lg = {'clf' : [MultiOutputClassifier(LogisticRegression(random_state=199))],
                     'clf__estimator__penalty' : ['l1', 'l2'],
                     'clf__estimator__C' : [0.1, 1, 3],
                     'clf__estimator__solver' : ['liblinear']}

params_options_empty = {}

In [110]:
from sklearn.model_selection import RandomizedSearchCV

#score = make_scorer(mean_absolute_error, greater_is_better=False)
jobs = -1
score = 'f1_micro'
def_cv = 3
verbosity_level=10

gs_local_w2v = GridSearchCV(estimator=pip_local_w2v,
            param_grid=params_options_local_w2v,
            scoring=score,
            cv=def_cv,
            n_jobs=jobs,
            verbose=verbosity_level)  

gs_glove = GridSearchCV(estimator=pip_glove,
            param_grid=params_options_glove,
            scoring=score,
            cv=def_cv,
            n_jobs=jobs,
            verbose=verbosity_level) 

gs_doc2vec = GridSearchCV(estimator=pip_doc2vec,
            param_grid=params_options_doc2vec,
            scoring=score,
            cv=def_cv,
            n_jobs=jobs,
            verbose=verbosity_level)

gs_cats_sim = GridSearchCV(estimator=pip_cats_sim,
            param_grid=params_options_cat_sim,
            scoring=score,
            cv=def_cv,
            n_jobs=jobs,
            verbose=verbosity_level) 

randomized_gs = RandomizedSearchCV(estimator=pip_local_w2v,
            param_distributions=params_options_local_w2v,
            scoring=score,
            cv=def_cv,
            n_jobs=jobs,
            verbose=verbosity_level,
            random_state=199,
            n_iter=5)

In [93]:
%%time

grids = [gs_local_w2v, gs_glove, gs_doc2vec, gs_cats_sim]

grid_dict = {
    0 : 'Feature: Local W2V',
    1 : 'Feature: Glove',
    2 : 'Feature: Doc2Vec',
    3 : 'Feature: Category Similarity'
}

# grids = [gs_all_feats_best_params]

# grid_dict = {
#     0 : 'Feature: All Features with Best Params',
# }

# grids = [gs_local_w2v]

# grid_dict = {
#     0 : 'Feature: Local W2V',
# }

print('Performing model optimizations...')
best_score = 0.0
best_clf = 0
best_gs = ''
overall_results_df = pd.DataFrame()
for idx, gs in enumerate(grids):
    print('\nEstimator: %s' % grid_dict[idx])
    # Fit grid search
    gs.fit(X_train, y_train)
    # Best params
    print('Best params: %s' % gs.best_params_)
    # Best training data accuracy
    print('Best training score: %.3f' % gs.best_score_)
    # Predict on test data with best params
    test_score = gs.score(X_test, y_test)
    # Test data accuracy of model with best params
    print('Test set score for best params: %.3f ' % test_score)
    
    # Track Grid Search results
    gs_results_df = pd.DataFrame(gs.cv_results_)
    gs_results_df['grid_id'] = grid_dict[idx]
    gs_results_df['best_model_test_score'] = test_score
    gs_results_df['param_set_order'] = np.arange(len(gs_results_df))
    
    overall_results_df = pd.concat([overall_results_df, gs_results_df])
    
    # Track best (highest test accuracy) model
    if test_score > best_score:
        best_score = test_score
        best_gs = gs
        best_clf = idx
print('\nClassifier with best test set accuracy: %s' % grid_dict[best_clf])

Performing model optimizations...

Estimator: Feature: Local W2V
Fitting 3 folds for each of 6 candidates, totalling 18 fits
[CV] clf=MultiOutputClassifier(estimator=RandomForestClassifier(max_depth=100,
                                                       min_samples_split=5,
                                                       n_estimators=50,
                                                       random_state=199)), local_w2v__num_dims=50 


/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:29: DeprecationWarning: Call to deprecated `__contains__` (Method will be removed in 4.0.0, use self.wv.__contains__() instead).
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:29: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:29: DeprecationWarning: Call to deprecated `__contains__` (Method will be removed in 4.0.0, use self.wv.__contains__() instead).
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:29: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


[CV]  clf=MultiOutputClassifier(estimator=RandomForestClassifier(max_depth=100,
                                                       min_samples_split=5,
                                                       n_estimators=50,
                                                       random_state=199)), local_w2v__num_dims=50, score=0.467, total= 3.0min
[CV] clf=MultiOutputClassifier(estimator=RandomForestClassifier(max_depth=100,
                                                       min_samples_split=5,
                                                       n_estimators=50,
                                                       random_state=199)), local_w2v__num_dims=50 


[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:  3.0min remaining:    0.0s
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:29: DeprecationWarning: Call to deprecated `__contains__` (Method will be removed in 4.0.0, use self.wv.__contains__() instead).
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:29: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:29: DeprecationWarning: Call to deprecated `__contains__` (Method will be removed in 4.0.0, use self.wv.__contains__() instead).
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:29: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


[CV]  clf=MultiOutputClassifier(estimator=RandomForestClassifier(max_depth=100,
                                                       min_samples_split=5,
                                                       n_estimators=50,
                                                       random_state=199)), local_w2v__num_dims=50, score=0.464, total= 3.0min
[CV] clf=MultiOutputClassifier(estimator=RandomForestClassifier(max_depth=100,
                                                       min_samples_split=5,
                                                       n_estimators=50,
                                                       random_state=199)), local_w2v__num_dims=50 


...

[Parallel(n_jobs=-1)]: Done   9 out of   9 | elapsed: 13.7min remaining:    0.0s


[CV]  cats_sim__word2vec_model=<gensim.models.keyedvectors.Word2VecKeyedVectors object at 0x7f5ffdf03cc0>, clf=MultiOutputClassifier(estimator=LogisticRegression(C=1, random_state=199,
                                                   solver='liblinear')), score=0.375, total=   9.0s
[CV] cats_sim__word2vec_model=<gensim.models.keyedvectors.Word2VecKeyedVectors object at 0x7f5ffdf03cc0>, clf=MultiOutputClassifier(estimator=LogisticRegression(C=1, random_state=199,
                                                   solver='liblinear')) 
[CV]  cats_sim__word2vec_model=<gensim.models.keyedvectors.Word2VecKeyedVectors object at 0x7f5ffdf03cc0>, clf=MultiOutputClassifier(estimator=LogisticRegression(C=1, random_state=199,
                                                   solver='liblinear')), score=0.369, total=   9.0s
[CV] cats_sim__word2vec_model=<gensim.models.keyedvectors.Word2VecKeyedVectors object at 0x7f5ffdf03cc0>, clf=MultiOutputClassifier(estimator=LogisticRegression(C=1, random_

[Parallel(n_jobs=-1)]: Done  18 out of  18 | elapsed: 22.1min finished


Best params: {'cats_sim__word2vec_model': <gensim.models.keyedvectors.Word2VecKeyedVectors object at 0x7f5fec1d4940>, 'clf': MultiOutputClassifier(estimator=RandomForestClassifier(max_depth=100,
                                                       min_samples_split=5,
                                                       n_estimators=50,
                                                       random_state=199))}
Best training score: 0.433
Test set score for best params: 0.435 

Classifier with best test set accuracy: Feature: Local W2V
CPU times: user 4h 25min 5s, sys: 30.7 s, total: 4h 25min 35s
Wall time: 4h 26min 43s


/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:47: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



In [94]:
overall_results_df

,best_model_test_score,grid_id,mean_fit_time,mean_score_time,mean_test_score,param_cats_sim__word2vec_model,param_clf,param_doc2vec__vector_size,param_glove__word2vec_model,param_local_w2v__num_dims,param_set_order,params,rank_test_score,split0_test_score,split1_test_score,split2_test_score,std_fit_time,std_score_time,std_test_score
0,0.475325,Feature: Local W2V,167.244666,13.906328,0.464909,NaN,MultiOutputClassifier(estimator=RandomForestCl...,NaN,NaN,50,0,{'clf': MultiOutputClassifier(estimator=Random...,3,0.467375,0.464434,0.462917,2.242656,0.354722,0.001851
1,0.475325,Feature: Local W2V,220.064234,13.668766,0.470284,NaN,MultiOutputClassifier(estimator=RandomForestCl...,NaN,NaN,100,1,{'clf': MultiOutputClassifier(estimator=Random...,2,0.471858,0.471068,0.467926,0.865947,0.301182,0.001698
2,0.475325,Feature: Local W2V,346.429916,14.566423,0.474161,NaN,MultiOutputClassifier(estimator=RandomForestCl...,NaN,NaN,300,2,{'clf': MultiOutputClassifier(estimator=Random...,1,0.476849,0.472814,0.472820,0.647241,0.282001,0.001901
3,0.475325,Feature: Local W2V,26.878744,12.008283,0.367483,NaN,MultiOutputClassifier(estimator=LogisticRegres...,NaN,NaN,50,3,{'clf': MultiOutputClassifier(estimator=Logist...,4,0.370103,0.365089,0.367257,0.206916,0.118321,0.002053
4,0.475325,Feature: Local W2V,29.203583,12.084064,0.367483,NaN,MultiOutputClassifier(estimator=LogisticRegres...,NaN,NaN,100,4,{'clf': MultiOutputClassifier(estimator=Logist...,4,0.370103,0.365089,0.367257,0.305883,0.120588,0.002053
5,0.475325,Feature: Local W2V,39.943756,12.376639,0.367483,NaN,MultiOutputClassifier(estimator=LogisticRegres...,NaN,NaN,300,5,{'clf': MultiOutputClassifier(estimator=Logist...,4,0.370103,0.365089,0.367257,0.131247,0.165002,0.002053
0,0.467183,Feature: Glove,154.236127,6.253713,0.452303,NaN,MultiOutputClassifier(estimator=RandomForestCl...,NaN,<gensim.models.keyedvectors.Word2VecKeyedVecto...,NaN,0,{'clf': MultiOutputClassifier(estimator=Random...,3,0.453860,0.449492,0.453557,0.849040,0.011945,0.001992
1,0.467183,Feature: Glove,208.462374,6.279925,0.457773,NaN,MultiOutputClassifier(estimator=RandomForestCl...,NaN,<gensim.models.keyedvectors.Word2VecKeyedVecto...,NaN,1,{'clf': MultiOutputClassifier(estimator=Random...,2,0.458837,0.457468,0.457015,1.141801,0.196657,0.000775
2,0.467183,Feature: Glove,355.494968,6.937809,0.465437,NaN,MultiOutputClassifier(estimator=RandomForestCl...,NaN,<gensim.models.keyedvectors.Word2VecKeyedVecto...,NaN,2,{'clf': MultiOutputClassifier(estimator=Random...,1,0.464921,0.466906,0.464483,6.199437,0.262951,0.001054
3,0.467183,Feature: Glove,17.073034,4.364227,0.433407,NaN,MultiOutputClassifier(estimator=LogisticRegres...,NaN,<gensim.models.keyedvectors.Word2VecKeyedVecto...,NaN,3,{'clf': MultiOutputClassifier(estimator=Logist...,6,0.437000,0.430560,0.432662,0.167471,0.049439,0.002682


In [96]:
results_feature_sets = overall_results_df

In [97]:
results_feature_sets.to_csv('f1micro_score_per_featset.csv', index=False)

In [104]:
gs_local_w2v.best_estimator_

Pipeline(steps=[('local_w2v',
                 TfidfEmbeddingTrainVectorizer(num_dims=300,
                                               word2vec_model=<gensim.models.word2vec.Word2Vec object at 0x7f5fc6f53048>)),
                ('clf',
                 MultiOutputClassifier(estimator=RandomForestClassifier(max_depth=100,
                                                                        min_samples_split=5,
                                                                        n_estimators=50,
                                                                        random_state=199)))])

In [105]:
gs_glove.best_estimator_

Pipeline(steps=[('glove',
                 TfidfEmbeddingTrainVectorizer(num_dims=300,
                                               word2vec_model=<gensim.models.keyedvectors.Word2VecKeyedVectors object at 0x7f5fba491f28>)),
                ('clf',
                 MultiOutputClassifier(estimator=RandomForestClassifier(max_depth=100,
                                                                        min_samples_split=5,
                                                                        n_estimators=50,
                                                                        random_state=199)))])

In [106]:
gs_doc2vec.best_estimator_

Pipeline(steps=[('doc2vec', Doc2VecTransformer(vector_size=50)),
                ('clf',
                 MultiOutputClassifier(estimator=RandomForestClassifier(max_depth=100,
                                                                        min_samples_split=5,
                                                                        n_estimators=50,
                                                                        random_state=199)))])

In [107]:
gs_cats_sim.best_estimator_

Pipeline(steps=[('cats_sim',
                 CategoriesSimilarity(categories_tokens=array([array(['related'], dtype='<U7'), array(['request'], dtype='<U7'),
       array(['offer'], dtype='<U5'),
       array(['aid', 'related'], dtype='<U7'),
       array(['medical', 'help'], dtype='<U7'),
       array(['medical', 'products'], dtype='<U8'),
       array(['search', 'and', 'rescue'], dtype='<U6'),
       array(['security'], dtyp...
       array(['cold'], dtype='<U4'),
       array(['other', 'weather'], dtype='<U7'),
       array(['direct', 'report'], dtype='<U6')], dtype=object),
                                      num_dims=300,
                                      word2vec_model=<gensim.models.keyedvectors.Word2VecKeyedVectors object at 0x7f5fb8629f60>)),
                ('clf',
                 MultiOutputClassifier(estimator=RandomForestClassifier(max_depth=100,
                                                                        min_samples_split=5,
                            

In [101]:
test = pd.read_csv('f1micro_score_per_featset.csv')
test[test['rank_test_score'] == 1]

,best_model_test_score,grid_id,mean_fit_time,mean_score_time,mean_test_score,param_cats_sim__word2vec_model,param_clf,param_doc2vec__vector_size,param_glove__word2vec_model,param_local_w2v__num_dims,param_set_order,params,rank_test_score,split0_test_score,split1_test_score,split2_test_score,std_fit_time,std_score_time,std_test_score
2,0.475325,Feature: Local W2V,346.429916,14.566423,0.474161,NaN,MultiOutputClassifier(estimator=RandomForestCl...,NaN,NaN,300.0,2,{'clf': MultiOutputClassifier(estimator=Random...,1,0.476849,0.472814,0.472820,0.647241,0.282001,0.001901
8,0.467183,Feature: Glove,355.494968,6.937809,0.465437,NaN,MultiOutputClassifier(estimator=RandomForestCl...,NaN,<gensim.models.keyedvectors.Word2VecKeyedVecto...,NaN,2,{'clf': MultiOutputClassifier(estimator=Random...,1,0.464921,0.466906,0.464483,6.199437,0.262951,0.001054
12,0.383461,Feature: Doc2Vec,306.331991,6.948815,0.384753,NaN,MultiOutputClassifier(estimator=RandomForestCl...,50.0,NaN,NaN,0,{'clf': MultiOutputClassifier(estimator=Random...,1,0.387285,0.381709,0.385264,2.333601,0.018058,0.002305
22,0.434835,Feature: Category Similarity,118.590605,4.333922,0.433137,<gensim.models.keyedvectors.Word2VecKeyedVecto...,MultiOutputClassifier(estimator=RandomForestCl...,NaN,NaN,NaN,4,{'cats_sim__word2vec_model': <gensim.models.ke...,1,0.436032,0.432530,0.430850,6.491331,0.236132,0.002159


In [108]:
#Test All Features together using their best params from GridSearch

In [114]:
pip_all_feats = Pipeline([
    ('features', FeatureUnion([
        ('local_w2v', TfidfEmbeddingTrainVectorizer(num_dims=300)),
        ('glove', TfidfEmbeddingTrainVectorizer(glove_300d_w2v)),
        ('doc2vec', Doc2VecTransformer(vector_size=50)),
        ('cats_sim', CategoriesSimilarity(categories_tokens=categories_tokens,word2vec_model=glove_300d_w2v))
    ])),
    ('clf', MultiOutputClassifier(RandomForestClassifier()))
])

In [115]:
params_options_all_feats = {'clf' : params_options_models_simple}

In [116]:
gs_all_feats_best_params = GridSearchCV(estimator=pip_all_feats,
                                  param_grid=params_options_all_feats,
                                 scoring=score,
                                 cv=def_cv,
                                 n_jobs=jobs,
            verbose=verbosity_level)

In [ ]:
grids = [gs_all_feats_best_params]

grid_dict = {
    0 : 'Feature: All Features with Best Params',
}

print('Performing model optimizations...')
all_feats_results = pd.DataFrame()
for idx, gs in enumerate(grids):
    print('\nEstimator: %s' % grid_dict[idx])
    # Fit grid search
    gs.fit(X_train, y_train)
    # Best params
    print('Best params: %s' % gs.best_params_)
    # Best training data accuracy
    print('Best training score: %.3f' % gs.best_score_)
    # Predict on test data with best params
    test_score = gs.score(X_test, y_test)
    # Test data accuracy of model with best params
    print('Test set score for best params: %.3f ' % test_score)
    
    # Track Grid Search results
    gs_results_df = pd.DataFrame(gs.cv_results_)
    gs_results_df['grid_id'] = grid_dict[idx]
    gs_results_df['best_model_test_score'] = test_score
    gs_results_df['param_set_order'] = np.arange(len(gs_results_df))
    
    all_feats_results = pd.concat([all_feats_results, gs_results_df])

In [118]:
all_feats_results

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_clf,params,split0_test_score,split1_test_score,split2_test_score,mean_test_score,std_test_score,rank_test_score,grid_id,best_model_test_score,param_set_order
0,660.303529,4.151322,28.056237,0.124641,MultiOutputClassifier(estimator=RandomForestCl...,{'clf': MultiOutputClassifier(estimator=Random...,0.471034,0.469648,0.469826,0.470169,0.000616,1,Feature: All Features with Best Params,0.473316,0
1,172.703791,1.489163,24.720033,0.489687,MultiOutputClassifier(estimator=LogisticRegres...,{'clf': MultiOutputClassifier(estimator=Logist...,0.450838,0.444178,0.446887,0.447301,0.002735,2,Feature: All Features with Best Params,0.473316,1


In [119]:
all_feats_results.to_csv('f1micro_score_all_feats.csv')

#### Final Decision between models

In [126]:
pip_local_w2v_final = Pipeline([
    ('local_w2v', TfidfEmbeddingTrainVectorizer(num_dims=300)),
    ('clf', MultiOutputClassifier(RandomForestClassifier()))
])

pip_all_feats_final = Pipeline([
    ('features', FeatureUnion([
        ('local_w2v', TfidfEmbeddingTrainVectorizer(num_dims=300)),
        ('glove', TfidfEmbeddingTrainVectorizer(glove_300d_w2v)),
        ('doc2vec', Doc2VecTransformer(vector_size=50)),
        ('cats_sim', CategoriesSimilarity(categories_tokens=categories_tokens,word2vec_model=glove_300d_w2v))
    ])),
    ('clf', MultiOutputClassifier(RandomForestClassifier()))
])

In [136]:
params_options_rf = {'clf' : [MultiOutputClassifier(RandomForestClassifier(random_state=199))],
                     'clf__estimator__n_estimators' : [50,100],
                     'clf__estimator__max_depth' : [50,100],
                     'clf__estimator__min_samples_split' : [2, 5]}

In [137]:
gs_local_w2v_final = GridSearchCV(estimator=pip_local_w2v_final,
            param_grid=params_options_rf,
            scoring=score,
            cv=def_cv,
            n_jobs=jobs,
            verbose=verbosity_level) 

gs_all_feats_final = GridSearchCV(estimator=pip_all_feats_final,
                                  param_grid=params_options_rf,
                                 scoring=score,
                                 cv=def_cv,
                                 n_jobs=jobs,
            verbose=verbosity_level)

In [138]:
grids_final = [gs_local_w2v_final]

grid_final_dict = {
    0 : 'Feature: Local W2V with Best Params',
    1 : 'Feature: All Features with Best Params'
}

print('Performing model optimizations...')
final_results = pd.DataFrame()
for idx, gs in enumerate(grids_final):
    print('\nEstimator: %s' % grid_dict[idx])
    # Fit grid search
    gs.fit(X_train, y_train)
    # Best params
    print('Best params: %s' % gs.best_params_)
    # Best training data accuracy
    print('Best training score: %.3f' % gs.best_score_)
    # Predict on test data with best params
    test_score = gs.score(X_test, y_test)
    # Test data accuracy of model with best params
    print('Test set score for best params: %.3f ' % test_score)
    
    # Track Grid Search results
    gs_results_df = pd.DataFrame(gs.cv_results_)
    gs_results_df['grid_id'] = grid_dict[idx]
    gs_results_df['best_model_test_score'] = test_score
    gs_results_df['param_set_order'] = np.arange(len(gs_results_df))
    
    final_results = pd.concat([final_results, gs_results_df])

Performing model optimizations...

Estimator: Feature: All Features with Best Params
Fitting 3 folds for each of 8 candidates, totalling 24 fits
[CV] clf=MultiOutputClassifier(estimator=RandomForestClassifier(random_state=199)), clf__estimator__max_depth=50, clf__estimator__min_samples_split=2, clf__estimator__n_estimators=50 


/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:29: DeprecationWarning: Call to deprecated `__contains__` (Method will be removed in 4.0.0, use self.wv.__contains__() instead).
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:29: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


KeyboardInterrupt: 

In [ ]:
param_grid_ = [
    {'features' : [MeanEmbeddingTrainVectorizer(),TfidfEmbeddingTrainVectorizer()],
     'features__word2vec_model' : [None,glove_50d_w2v,glove_100d_w2v,glove_300d_w2v],
     'features__num_dims' : [50,100,300]},
    {'features' : [Doc2VecTransformer()],
     'features__vector_size' : [50,100,300]},
    {'features' : [CategoriesSimilarity(glove_50d_w2v,categories_tokens)],
     'features__word2vec_model' : [glove_50d_w2v,glove_100d_w2v,glove_300d_w2v]},
    {'features' : [MeanEmbeddingTrainVectorizer(),TfidfEmbeddingTrainVectorizer()],
     'features__word2vec_model' : [None,glove_50d_w2v,glove_100d_w2v,glove_300d_w2v],
     'features__num_dims' : [50,100,300]},
    {'classifier' : [MultiOutputClassifier(LogisticRegression(random_state=199))],
     'classifier__estimator__penalty' : ['l1', 'l2'],
     'classifier__estimator__C' : [0.1, 1, 3],
     'classifier__estimator__solver' : ['liblinear']},
    {'classifier' : [MultiOutputClassifier(RandomForestClassifier(random_state=199))],
    'classifier__estimator__n_estimators' : [10,50,250],
    'classifier__estimator__max_depth' : [10, 50, 100],
    'classifier__estimator__min_samples_split' : [1, 5, 10]}
]

In [92]:
from sklearn import svm
from sklearn.metrics import make_scorer, mean_absolute_error

pip_local_w2v_tfidf_naive = Pipeline([
    ('local_w2v_tfidf', TfidfEmbeddingTrainVectorizer()),
    ('clf', MultiOutputClassifier(GaussianNB()))
])

pip_local_w2v_tfidf_rf = Pipeline([
    ('local_w2v_tfidf', TfidfEmbeddingTrainVectorizer()),
    ('clf', MultiOutputClassifier(RandomForestClassifier(random_state=199)))
])

pip_glove_tfidf_naive = Pipeline([
    ('glove_tfidf', TfidfEmbeddingTrainVectorizer(glove_300d_w2v)),
    ('clf', MultiOutputClassifier(GaussianNB()))
])

pip_glove_tfidf_rf = Pipeline([
    ('glove_tfidf', TfidfEmbeddingTrainVectorizer(glove_300d_w2v)),
    ('clf', MultiOutputClassifier(RandomForestClassifier(random_state=199)))
])

pip_glove_tfidf_lg = Pipeline([
    ('glove_tfidf', TfidfEmbeddingTrainVectorizer(glove_300d_w2v)),
    ('clf', MultiOutputClassifier(LogisticRegression(random_state=199)))
])

pip_doc2vec_naive = Pipeline([
    ('doc2vec', Doc2VecTransformer()),
    ('clf', MultiOutputClassifier(GaussianNB()))
])

pip_doc2vec_rf = Pipeline([
    ('doc2vec', Doc2VecTransformer()),
    ('clf', MultiOutputClassifier(RandomForestClassifier(random_state=199)))
])

pip_cats_sim_naive = Pipeline([
    ('cats_sim', CategoriesSimilarity(glove_300d_w2v, categories_tokens)),
    ('clf', MultiOutputClassifier(GaussianNB()))
])

pip_cats_sim_rf = Pipeline([
    ('cats_sim', CategoriesSimilarity(glove_300d_w2v, categories_tokens)),
    ('clf', MultiOutputClassifier(RandomForestClassifier(random_state=199)))
])

pip_all_feats_naive = Pipeline([
    ('features', FeatureUnion([
        ('local_w2v_tfidf', TfidfEmbeddingTrainVectorizer()),
        ('glove_tfidf', TfidfEmbeddingTrainVectorizer(glove_300d_w2v)),
        ('doc2vec', Doc2VecTransformer()),
        ('cats_sim', CategoriesSimilarity(glove_300d_w2v, categories_tokens))
    ])),
    ('clf', MultiOutputClassifier(GaussianNB()))
])

pip_all_feats_rf = Pipeline([
    ('features', FeatureUnion([
        ('local_w2v_tfidf', TfidfEmbeddingTrainVectorizer()),
        ('glove_tfidf', TfidfEmbeddingTrainVectorizer(glove_300d_w2v)),
        ('doc2vec', Doc2VecTransformer()),
        ('cats_sim', CategoriesSimilarity(glove_300d_w2v, categories_tokens))
    ])),
    ('clf', MultiOutputClassifier(RandomForestClassifier(random_state=199)))
])

In [94]:
grid_params_empty = [{}]

# grid_params_rf = [{'clf__criterion': ['gini', 'entropy'],
#         'clf__min_samples_leaf': param_range,
#         'clf__max_depth': param_range,
#         'clf__min_samples_split': param_range[1:]}]

# grid_params_svm = [{'clf__kernel': ['linear', 'rbf'], 
#         'clf__C': param_range}]

# Construct grid searches
jobs = -1

#score = make_scorer(mean_absolute_error, greater_is_better=False)
score = 'f1_micro'

def_cv = 3

gs_local_w2v_tfidf_naive = GridSearchCV(estimator=pip_local_w2v_tfidf_naive,
            param_grid=grid_params_empty,
            scoring=score,
            cv=def_cv,
            n_jobs=jobs)  

gs_local_w2v_tfidf_rf = GridSearchCV(estimator=pip_local_w2v_tfidf_rf,
            param_grid=grid_params_empty,
            scoring=score,
            cv=def_cv,
            n_jobs=jobs)

gs_glove_tfidf_naive = GridSearchCV(estimator=pip_glove_tfidf_naive,
            param_grid=grid_params_empty,
            scoring=score,
            cv=def_cv,
            n_jobs=jobs) 

gs_glove_tfidf_rf = GridSearchCV(estimator=pip_glove_tfidf_rf,
            param_grid=grid_params_empty,
            scoring=score,
            cv=def_cv,
            n_jobs=jobs)

gs_glove_tfidf_lg = GridSearchCV(estimator=pip_glove_tfidf_lg,
            param_grid=grid_params_empty,
            scoring=score,
            cv=def_cv,
            n_jobs=jobs)


gs_doc2vec_naive = GridSearchCV(estimator=pip_doc2vec_naive,
            param_grid=grid_params_empty,
            scoring=score,
            cv=def_cv,
            n_jobs=jobs)

gs_doc2vec_rf = GridSearchCV(estimator=pip_doc2vec_rf,
            param_grid=grid_params_empty,
            scoring=score,
            cv=def_cv,
            n_jobs=jobs)

gs_cats_sim_naive = GridSearchCV(estimator=pip_cats_sim_naive,
            param_grid=grid_params_empty,
            scoring=score,
            cv=def_cv,
            n_jobs=jobs) 

gs_cats_sim_rf = GridSearchCV(estimator=pip_cats_sim_rf,
            param_grid=grid_params_empty,
            scoring=score,
            cv=def_cv,
            n_jobs=jobs) 

gs_all_feats_naive = GridSearchCV(estimator=pip_all_feats_naive,
                                  param_grid=grid_params_empty,
                                 scoring=score,
                                 cv=def_cv,
                                 n_jobs=jobs)

gs_all_feats_rf = GridSearchCV(estimator=pip_all_feats_rf,
                                  param_grid=grid_params_empty,
                                 scoring=score,
                                 cv=def_cv,
                                 n_jobs=jobs)

# grids = [gs_local_w2v_tfidf_rf, gs_glove_tfidf_rf, 
#          gs_doc2vec_rf, gs_cats_sim_rf,
#          gs_all_feats_rf]

# grid_dict = {
#     0 : 'Local Word2Vec - TF-IDF - Random Forest',
#     1 : 'Glove 300d - TF-IDF - Random Forest',
#     2 : 'Doc2Vec - Random Forest',
#     3 : 'Categories Similarity - Random Forest',
#     4 : 'All Features - Random Forest'
# }

# grids = [gs_local_w2v_tfidf_naive, gs_glove_tfidf_naive, 
#          gs_doc2vec_naive, gs_cats_sim_naive,
#          gs_all_feats_naive]

# grid_dict = {
#     0 : 'Local Word2Vec - TF-IDF - Naive Bayes',
#     1 : 'Glove 300d - TF-IDF - Naive Bayes',
#     2 : 'Doc2Vec - Naive Bayes',
#     3 : 'Categories Similarity - Naive Bayes',
#     4 : 'All Features - Naive Bayes'
# }

grids = [gs_glove_tfidf_lg]

grid_dict = {
    0 : 'Glove 300d - TF-IDF - Logistic Regression',
}

In [95]:
%%time

print('Performing model optimizations...')
best_score = 0.0
best_clf = 0
best_gs = ''
overall_results_df = pd.DataFrame()
for idx, gs in enumerate(grids):
    print('\nEstimator: %s' % grid_dict[idx])
    # Fit grid search
    gs.fit(X_train, y_train)
    # Best params
    print('Best params: %s' % gs.best_params_)
    # Best training data accuracy
    print('Best training score: %.3f' % gs.best_score_)
    # Predict on test data with best params
    test_score = gs.score(X_test, y_test)
    # Test data accuracy of model with best params
    print('Test set score for best params: %.3f ' % test_score)
    
    # Track Grid Search results
    gs_results_df = pd.DataFrame(gs.cv_results_)
    gs_results_df['grid_id'] = grid_dict[idx]
    gs_results_df['best_model_test_score'] = test_score
    
    overall_results_df = pd.concat([overall_results_df, gs_results_df])
    
    # Track best (highest test accuracy) model
    if test_score > best_score:
        best_score = test_score
        best_gs = gs
        best_clf = idx
print('\nClassifier with best test set accuracy: %s' % grid_dict[best_clf])

Performing model optimizations...

Estimator: Glove 300d - TF-IDF - Logistic Regression
Best params: {}
Best training score: 0.444
Test set score for best params: 0.447 

Classifier with best test set accuracy: Glove 300d - TF-IDF - Logistic Regression
CPU times: user 6min 28s, sys: 4.3 s, total: 6min 32s
Wall time: 6min 34s


/opt/conda/lib/python3.6/site-packages/sklearn/utils/deprecation.py:122: FutureWarning: You are accessing a training score ('split0_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
/opt/conda/lib/python3.6/site-packages/sklearn/utils/deprecation.py:122: FutureWarning: You are accessing a training score ('split1_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
/opt/conda/lib/python3.6/site-packages/sklearn/utils/deprecation.py:122: FutureWarning: You are accessing a training score ('split2_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
/opt/conda/lib/python3.6/site-packages/sklearn/utils/deprecation.py

In [73]:
overall_results_df

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,params,split0_test_score,split1_test_score,split2_test_score,mean_test_score,std_test_score,rank_test_score,split0_train_score,split1_train_score,split2_train_score,mean_train_score,std_train_score,grid_id,best_model_test_score
0,24.193751,1.243320,9.702975,0.216793,{},0.216412,0.216126,0.212304,0.214947,0.001873,1,0.218802,0.211861,0.217092,0.215918,0.002952,Local Word2Vec - TF-IDF - Naive Bayes,0.209587
0,8.761650,0.157636,4.028346,0.026570,{},0.227605,0.232018,0.221844,0.227156,0.004165,1,0.229607,0.229266,0.227849,0.228907,0.000761,Glove 300d - TF-IDF - Naive Bayes,0.225530
0,5.382411,0.169216,2.269197,0.052283,{},0.374412,0.368413,0.370370,0.371065,0.002498,1,0.371911,0.373925,0.372439,0.372759,0.000852,Doc2Vec - Naive Bayes,0.368967
0,4.361952,0.024142,2.160518,0.002270,{},0.218708,0.223602,0.218829,0.220380,0.002279,1,0.221607,0.220599,0.221507,0.221238,0.000453,Categories Similarity - Naive Bayes,0.219315
0,46.058670,2.215212,18.078570,0.056807,{},0.223004,0.225169,0.217339,0.221837,0.003301,1,0.224831,0.222482,0.222858,0.223390,0.001030,All Features - Naive Bayes,0.219720


In [74]:
results_lg_f1micro_score = overall_results_df

In [75]:
results_lg_f1micro_score.to_csv('naive_f1micro_score.csv', index=False)

# Save best grid search pipeline to file
dump_file = 'best_gs_pipeline.pkl'
joblib.dump(best_gs, dump_file, compress=1)
print('\nSaved %s grid search pipeline to file: %s' % (grid_dict[best_clf], dump_file))

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

X_train.shape

X_train

# train classifier
pipeline.fit(X_train, y_train)

# predict on test data
y_pred = pipeline.predict(X_test)

print(X_test.shape)

print(y_pred.shape)
y_pred

y_pred[:,0]

y_test

for category_idx in range(y_pred.shape[1]):
    print(classification_report(y_pred=y_pred[:,category_idx],y_true=y_test[:,category_idx], labels=[0,1], target_names=[df.columns[4+category_idx] + '-0',df.columns[4+category_idx] + '-1']))

### 6. Improve your model
Use grid search to find better parameters. 

pipeline.get_params().keys()

parameters = {
        'multi_clf__estimator__n_estimators': [20,50],
        'multi_clf__estimator__max_depth': [50, 100]
    }

cv = GridSearchCV(pipeline, param_grid=parameters, verbose=10)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

# train classifier
cv.fit(X_train, y_train)

# predict on test data
y_pred = cv.predict(X_test)

cv.best_params_

for category_idx in range(y_pred.shape[1]):
    print(classification_report(y_pred=y_pred[:,category_idx],y_true=y_test[:,category_idx], labels=[0,1], target_names=[df.columns[4+category_idx] + '-0',df.columns[4+category_idx] + '-1']))

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

# Output a pickle file for the model
joblib.dump(cv, 'classifier.pkl') 

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.